In [8]:
import os
import shutil

def rename_and_copy_images(input_path, output_path, prefix='sd_114_'):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Loop through all files in the input directory
    for file in os.listdir(input_path):
        # Check if the file is a TIFF image
        if file.endswith('.tiff'):
            # Define the old and new file paths
            old_path = os.path.join(input_path, file)
            new_filename = prefix + file
            new_path = os.path.join(output_path, new_filename)
            
            # Copy and rename the file to the output directory
            shutil.copy(old_path, new_path)
            print(f"Copied and renamed: {old_path} -> {new_path}")

# Example usage
input_path = r'C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated'
output_path = r'C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated'
rename_and_copy_images(input_path, output_path)


Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B02-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B02-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B03-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B03-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B04-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B04-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B05-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B05-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B06-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B06-T01.tiff


In [11]:
import tifffile as tiff

def check_image_layers(image_dir):
    # Get all TIFF images in the directory
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')]
    
    # Loop through each image and check its layers
    for img_path in image_files:
        image = tiff.imread(img_path)
        
        # Check if the image has exactly 3 layers (channels)
        if image.shape[0] < 3:
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")

# Example usage
image_dir = r"../../Day10_drugscreened&singledose_untreated"
check_image_layers(image_dir)

In [19]:
import os
import numpy as np
import tifffile as tiff

def replace_with_3_sharpest_layers(image_dir):
    # Get all TIFF images in the directory
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')]
    
    # Process each image
    for img_path in image_files:
        # Read the image
        image = tiff.imread(img_path)
        
        # Check the number of layers and dimensions of the image
        if image.ndim == 2:  # Image is 2D, likely a single-channel image
            print(f"Warning: Image {img_path} is single-channel (dimensions: {image.shape}).")
            continue  # Skip processing for this image
        elif image.shape[0] < 3:  # Fewer than 3 layers
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")
            continue  # Skip processing for this image

        # If the image has more than 3 layers, proceed
        if image.shape[0] > 3:
            print(f"Processing {img_path} with {image.shape[0]} layers.")
            
            # Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
            image_normalized = image.astype(np.float32) / 65535.0
            
            # Calculate sharpness for each layer
            sharpness_scores = []
            for i in range(image.shape[0]):
                layer = image_normalized[i]
                gy, gx = np.gradient(layer)  # Compute gradients
                gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
                sharpness = np.average(gnorm)  # Average sharpness
                sharpness_scores.append((sharpness, i))  # Store with index
            
            # Sort layers by sharpness score and select the top 3 sharpest layers
            sharpest_layers = sorted(sharpness_scores, reverse=True, key=lambda x: x[0])[:3]
            top_3_indices = [index for _, index in sharpest_layers]
            
            # Create a new image with only the 3 sharpest layers
            new_image = image[top_3_indices, :, :]
            
            # Save the new image to replace the original
            tiff.imwrite(img_path, new_image.astype(np.uint16))
            print(f"Replaced {img_path} with the 3 sharpest layers.")

# Example usage
image_dir = r"../../Day10_drugscreened&singledose_untreated"
replace_with_3_sharpest_layers(image_dir)


In [16]:
path = r'C:\Users\k54739\Bibi_new_thesis\thesis\Day10_drugscreened&singledose_untreated\ds_41_E11-T01.tiff'
imageds = tiff.imread(path)

In [17]:
imageds.shape

(2054, 2456)